# 查询转换
从修改用户查询的方面提高检索的性能
在不依赖langchain的情况下完成三种查询转换方法增强rag的检索性能 


**查询重写(query rewrite)：** 改写用户查询更加具体详细
**回退提示(step-back prompting)：** 生成广泛查询
**子查询分解(sub-query decomposition)：** 将复杂查询拆分

In [ ]:
import fitz
import numpy as np
import json
from zhipuai import ZhipuAI
from tqdm import tqdm
import re

pdf_path = "AI_Information.en.zh-CN.pdf"
API_KEY = "f532b4bd71324c1ca2fd7e22d4eb41da.4Sfpr8wE6qEVV5BY"
# 设置客户端,处理文本
client = ZhipuAI(api_key = API_KEY) 

# model
llm_model = "glm-4"
embedding_model="embedding-3"

**查询重写(query rewrite)：**

In [ ]:
def rewrite_query(original_query):
    """
    重写查询，提高查询的具体性和详细程度。提高检索准确率

    Args:
        original_query (str): 原始查询
        model (str): 用于查询重写的模型

    Returns:
        str: 重写后的查询
    """
    # 定义系统提示
    system_prompt = "你是一个专注于优化搜索查询的专业的查询重写助手，你的任务是将用户的查询转换为更具体和详细的形式，以提高检索性能。请确保重写后的查询保留原始查询的核心信息，但更加明确和具体。"
    
    # 定义用户提示
    user_prompt = f"""
    请优化以下原始查询，使其满足：
    1. 增强查询的具体性和详细程度
    2. 包含有助于获取准确信息的相关术语和核心概念
    3. 不得出现违反原始查询或者原始查询没有的内容

    原始查询：{original_query}

    优化后的查询：
    """

    response = client.chat.completions.create(
        model=llm_model,
        temperature=0.0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    return response.choices[0].message.content.strip()

**回退提示（step-back prompting）**

In [ ]:
def generate_step_back_query(original_query):
    """
    生成一个更广泛的“回退”查询以检索更宽泛的上下文信息。

    Args:
        original_query (str): 原始用户查询
        model (str): 用于生成回退查询的模型

    Returns:
        str: 回退查询
    """
    # 定义系统提示，以指导AI助手的行为
    system_prompt = "你是一个专注于优化查询策略的AI助手。你的任务是使用回退查询策略将特定查询转化为更宽泛、更通用的版本，以帮助检索原始查询的相关背景信息。"

    # 定义用户提示，包含要概括的原始查询
    user_prompt = f"""
    请基于以下具体原始查询生成更通用的版本，要求：
    1. 扩大查询范围以涵盖背景信息
    2. 包含潜在相关领域的关键概念
    3. 保持语义完整性
    4. 保留原始查询的核心信息

    原始查询: {original_query}

    通用化查询：
    """

    # 使用指定的模型生成回退查询
    response = client.chat.completions.create(
        model=llm_model,
        temperature=0.1, # 增加多样性
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    return response.choices[0].message.content.strip()


**子查询分解（sub-query decomposition）**

In [ ]:
def decompose_query(original_query, num_subqueries=4):
    """
    分解原始查询为多个更简单的子查询，实现全面检索。

    Args:
        original_query (str): 原始查询
        num_subqueries (int): 子查询数量
        model (str): 用于查询分解的模型

    Returns:
        List[str]: 子查询的列表
    """
    # 定义系统提示，指导AI助手的行为
    system_prompt = "您是一个专门负责分解复杂问题的AI助手。您的任务是将复杂的查询拆解成更简单的子问题，这些子问题的答案组合起来能够解决原始查询。"

    # 使用需要分解的原始查询定义用户提示
    user_prompt = f"""
    将以下复杂查询分解为{num_subqueries}个更简单的子问题。每个子问题应聚焦原始问题的不同方面。

    原始查询: {original_query}

    请生成{num_subqueries}个子问题，需要保证这些问题能完整表达原始问题的信息。每个问题单独一行，按以下格式：
    1. [第一个子问题]
    2. [第二个子问题]
    依此类推...
    """

    # 使用指定模型生成子查询
    response = client.chat.completions.create(
        model=llm_model,
        temperature=0.2,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    # 处理响应以提取子查询
    content = response.choices[0].message.content.strip()

    pattern = r'^\d+\.\s*(.*)'
    return [re.match(pattern, line).group(1) for line in content.split('\n') if line.strip()]


In [ ]:
# 测试
original_query = "人工智能 (AI) 对工作自动化和就业有何影响？"

# 查询转换
print("原始查询:", original_query)

# 查询重写
rewritten_query = rewrite_query(original_query)
print("\n1. 重写查询:")
print(rewritten_query)

# 回退查询
step_back_query = generate_step_back_query(original_query)
print("\n2. 回退查询:")
print(step_back_query)

# 子查询分解
sub_queries = decompose_query(original_query, num_subqueries=4)
print("\n3. 子查询:")
for i, query in enumerate(sub_queries, 1):
    print(f"   ({i}) {query}")

原始查询: 人工智能 (AI) 对工作自动化和就业有何影响？

1. 重写查询:
人工智能（AI）如何具体影响不同行业的工作流程自动化及其对当前就业市场的影响与未来趋势分析？

2. 回退查询:
技术进步，特别是人工智能和自动化，如何影响现代劳动力市场与职业发展？

3. 子查询:
   (1) 人工智能目前在工作自动化中扮演了哪些角色？
   (2) 人工智能推动自动化后，哪些类型的工作岗位可能会受到影响？
   (3) 人工智能对就业市场有哪些潜在的正面影响？
   (4) 如何应对人工智能带来的就业挑战，有哪些策略可以减轻其负面影响？


**构建向量存储**

In [ ]:
class SimpleVectorStore:
    def __init__(self):
        """
        初始化向量存储。
        """
        self.vectors = []  # 嵌入向量列表
        self.texts = []  # 原始文本列表
        self.metadata = []  # 每个文本元数据的列表

    def add_item(self, text, embedding, metadata=None):
        """
        向向量存储中添加一个项目。

        Args:
        text (str): 原始文本。
        embedding (List[float]): 嵌入向量。
        metadata (dict, 可选): 额外的元数据。
        """
        self.vectors.append(np.array(embedding))  # 将嵌入转换为numpy数组并添加到向量列表中
        self.texts.append(text)  # 将原始文本添加到文本列表中
        self.metadata.append(metadata or {})  # 添加元数据到元数据列表中，如果没有提供则使用空字典

    def similarity_search(self, query_embedding, k=5):
        """
        查找与查询嵌入最相似的项目。

        Args:
        query_embedding (List[float]): 查询嵌入向量。
        k (int): 返回的结果数量。

        Returns:
        List[Dict]: 包含文本和元数据的前k个最相似项。
        """
        if not self.vectors:
            return []  # 如果没有存储向量，则返回空列表

        query_vector = np.array(query_embedding)

        similarities = []
        for i, vector in enumerate(self.vectors):
            # 计算查询向量与存储向量之间的余弦相似度
            similarity = np.dot(query_vector, vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
            similarities.append((i, similarity))  # 添加索引和相似度分数

        # 按相似度排序
        similarities.sort(key=lambda x: x[1], reverse=True)

        # 返回前k个结果
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],  # 添加对应的文本
                "metadata": self.metadata[idx],  # 添加对应的元数据
                "similarity": score  # 添加相似度分数
            })

        return results  # 返回前k个最相似项的列表


**RAG构建**

In [22]:
def create_embeddings(text):
    """
    使用Embedding模型为给定文本创建嵌入向量。

    Args:
    text (str): 要创建嵌入向量的输入文本。

    Returns:
    List[float]: 嵌入向量。
    """
    # 通过将字符串输入转换为列表来处理字符串和列表输入
    input_text = text if isinstance(text, list) else [text]

    # 使用指定的模型为输入文本创建嵌入向量
    response = client.embeddings.create(
        model=embedding_model,
        input=input_text
    )

    # 如果输入是字符串，仅返回第一个嵌入向量
    if isinstance(text, str):
        return response.data[0].embedding

    # 否则，将所有嵌入向量作为向量列表返回
    return [item.embedding for item in response.data]

def extract_text_from_pdf(pdf_path):
    # 打开 PDF 文件
    mypdf = fitz.open(pdf_path)
    all_text = ""  # 初始化一个空字符串以存储提取的文本

    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]
        text = page.get_text("text")  # 从页面中提取文本
        all_text += text  # 将提取的文本追加到 all_text 字符串中

    return all_text  # 返回提取的文本

def chunk_text(text, n, overlap):
    """
    将文本分割为重叠的块

    Args:
    text (str): 要分割的文本
    n (int): 每个块的字符数
    overlap (int): 块之间的重叠字符数

    Returns:
    List[str]: 文本块列表
    """
    chunks = []  #
    for i in range(0, len(text), n - overlap):
        # 添加从当前索引到索引 + 块大小的文本块
        chunks.append(text[i:i + n])

    return chunks 

def process_document(pdf_path, chunk_size=1000, chunk_overlap=200):
    """
    为RAG处理文档。

    Args:
    pdf_path (str): PDF文件的路径。
    chunk_size (int): 每个文本块的大小（以字符为单位）。
    chunk_overlap (int): 文本块之间的重叠大小（以字符为单位）。

    Returns:
    SimpleVectorStore: 包含文档文本块及其嵌入向量的向量存储。
    """
    print("从PDF中提取文本...")
    extracted_text = extract_text_from_pdf(pdf_path)  # 调用函数提取PDF中的文本

    print("分割文本...")
    chunks = chunk_text(extracted_text, chunk_size, chunk_overlap)  # 将提取的文本分割为多个块
    print(f"创建了 {len(chunks)} 个文本块")

    print("为文本块创建嵌入向量...")
    # 为了提高效率，一次性为所有文本块创建嵌入向量
    chunk_embeddings = create_embeddings(chunks)

    # 创建向量存储
    store = SimpleVectorStore()

    # 将文本块添加到向量存储中
    for i, (chunk, embedding) in enumerate(zip(chunks, chunk_embeddings)):
        store.add_item(
            text=chunk,  # 文本内容
            embedding=embedding,  # 嵌入向量
            metadata={"index": i, "source": pdf_path}  # 元数据，包括索引和源文件路径
        )

    print(f"向向量存储中添加了 {len(chunks)} 个文本块")
    return store


**基于查询转换的检索增强生成**

In [23]:
def transformed_search(query, vector_store, transformation_type, top_k=3):
    """
    使用转换后的查询进行搜索。

    Args:
        query (str): 原始查询
        vector_store (SimpleVectorStore): 用于搜索的向量存储
        transformation_type (str): 转换类型 ('rewrite', 'step_back', 或 'decompose')
        top_k (int): 返回的结果数量

    Returns:
        List[Dict]: 搜索结果
    """
    print(f"转换类型: {transformation_type}")
    print(f"原始查询: {query}")

    results = []

    if transformation_type == "rewrite":
        # 查询重写
        transformed_query = rewrite_query(query)
        print(f"重写后的查询: {transformed_query}")

        # 为转换后的查询创建嵌入向量
        query_embedding = create_embeddings(transformed_query)

        # 使用重写后的查询进行搜索
        results = vector_store.similarity_search(query_embedding, k=top_k)

    elif transformation_type == "step_back":
        # 回退提示
        transformed_query = generate_step_back_query(query)
        print(f"后退查询: {transformed_query}")

        # 为转换后的查询创建嵌入向量
        query_embedding = create_embeddings(transformed_query)

        # 使用回退查询进行搜索
        results = vector_store.similarity_search(query_embedding, k=top_k)

    elif transformation_type == "decompose":
        # 子查询分解
        sub_queries = decompose_query(query)
        print("分解为子查询:")
        for i, sub_q in enumerate(sub_queries, 1):
            print(f"{i}. {sub_q}")

        # 为所有子查询创建嵌入向量
        sub_query_embeddings = create_embeddings(sub_queries)

        # 使用每个子查询进行搜索并合并结果
        all_results = []
        for i, embedding in enumerate(sub_query_embeddings):
            sub_results = vector_store.similarity_search(embedding, k=2)  # 每个子查询获取较少的结果
            all_results.extend(sub_results)

        # 去重（保留相似度最高的结果）
        seen_texts = {}
        for result in all_results:
            text = result["text"]
            if text not in seen_texts or result["similarity"] > seen_texts[text]["similarity"]:
                seen_texts[text] = result

        # 按相似度排序并取前 top_k 个结果
        results = sorted(seen_texts.values(), key=lambda x: x["similarity"], reverse=True)[:top_k]

    else:
        # 普通搜索（无转换）
        query_embedding = create_embeddings(query)
        results = vector_store.similarity_search(query_embedding, k=top_k)

    return results


**带转换查询的回答**

In [ ]:
def generate_response(query, context):
    """
    根据查询和检索到的上下文生成响应。

    Args:
        query (str): 用户查询
        context (str): 检索到的上下文
    Returns:
        str: 生成的响应
    """
    # 定义系统提示以指导AI助手的行为
    system_prompt = "您是一个乐于助人的AI助手。请仅根据提供的上下文来回答用户的问题。如果在上下文中找不到答案，请直接说'没有足够的信息'。"

    # 定义包含上下文和查询的用户提示
    user_prompt = f"""
        上下文内容:
        {context}

        问题: {query}

        请基于上述上下文内容提供一个全面详尽的答案。
    """

    # 使用指定的模型生成响应
    response = client.chat.completions.create(
        model=llm_model,
        temperature=0,  
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    return response.choices[0].message.content.strip()


**带查询转换的完整rag**

In [25]:
def rag_with_query_transformation(pdf_path, query, transformation_type=None):
    """
    运行完整的RAG管道，并可选地进行查询转换。

    Args:
        pdf_path (str): PDF文档的路径
        query (str): 用户查询
        transformation_type (str): 转换类型（None、'rewrite'、'step_back' 或 'decompose'）

    Returns:
        Dict: 包括原始查询、转换后的查询、上下文和回答的结果
    """
    # 处理文档以创建向量存储
    vector_store = process_document(pdf_path)

    # 应用查询转换并搜索
    if transformation_type:
        # 使用转换后的查询进行搜索
        results = transformed_search(query, vector_store, transformation_type)
    else:
        # 不进行转换，执行常规搜索
        query_embedding = create_embeddings(query)
        results = vector_store.similarity_search(query_embedding, k=3)

    # 从搜索结果中组合上下文
    context = "\n\n".join([f"段落 {i+1}:\n{result['text']}" for i, result in enumerate(results)])

    # 根据查询和组合后的上下文生成响应
    response = generate_response(query, context)

    # 返回结果，包括原始查询、转换类型、上下文和响应
    return {
        "original_query": query,
        "transformation_type": transformation_type,
        "context": context,
        "response": response
    }


**eval**

In [26]:
def compare_responses(results, reference_answer):
    """
    比较不同查询转换技术生成的响应。

    Args:
        results (Dict): 不同转换技术生成的结果
        reference_answer (str): 用于比较的参考答案
    """
    # 定义系统提示以指导AI助手的行为
    system_prompt = """您是RAG系统评估专家。您的任务是比较使用不同查询转换技术生成的回答，并确定哪种技术生成的回答最接近参考答案。"""

    # 准备包含参考答案和每种技术生成的响应的比较文本
    comparison_text = f"""参考答案: {reference_answer}\n\n"""

    for technique, result in results.items():
        comparison_text += f"{technique.capitalize()} 查询回答:\n{result['response']}\n\n"

    # 定义用户提示，包含比较文本
    user_prompt = f"""
    {comparison_text}

    请将不同查询转换技术生成的回答与参考答案进行对比分析。

    针对每种技术（原始查询、重写查询、回退查询、问题分解）进行评判：
    1. 根据准确性、完整性和相关性给出1-10分的评分
    2. 分别指出该技术生成回答的优点和不足

    最后对所有技术进行排序，并说明整体表现最佳的技术及其优势原因。
    """

    # 使用指定模型生成评估响应
    response = client.chat.completions.create(
        model=llm_model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    # 打印评估结果
    print("\n===== 评估结果 =====")
    print(response.choices[0].message.content)
    print("====================")


In [27]:
def evaluate_transformations(pdf_path, query, reference_answer=None):
    """
    评估同一查询的不同转换技术。

    Args:
        pdf_path (str): PDF文档的路径
        query (str): 要评估的查询
        reference_answer (str): 可选的参考答案用于比较

    Returns:
        Dict: 评估结果
    """
    # 定义要评估的转换技术
    transformation_types = [None, "rewrite", "step_back", "decompose"]
    results = {}

    # 使用每种转换技术运行RAG
    for transformation_type in transformation_types:
        type_name = transformation_type if transformation_type else "original"
        print(f"\n===== 使用 {type_name} 查询运行 RAG =====")

        # 获取当前转换类型的结果
        result = rag_with_query_transformation(pdf_path, query, transformation_type)
        results[type_name] = result

        # 打印当前转换类型的响应
        print(f"使用 {type_name} 查询的响应:")
        print(result["response"])
        print("=" * 50)

    # 如果提供了参考答案，则比较结果
    if reference_answer:
        compare_responses(results, reference_answer)

    return results


In [28]:
# Load the validation data from a JSON file
with open('val.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Extract the first query from the validation data
query = data[3]['question']

# Extract the reference answer from the validation data
reference_answer = data[0]['ideal_answer']

# Run evaluation
evaluation_results = evaluate_transformations(pdf_path, query, reference_answer)
print(evaluation_results)


===== 使用 original 查询运行 RAG =====
从PDF中提取文本...
分割文本...
创建了 13 个文本块
为文本块创建嵌入向量...
向向量存储中添加了 13 个文本块
使用 original 查询的响应:
人工智能为个性化医疗做出了多方面的贡献。首先，人工智能通过分析个体患者数据，能够预测治疗反应并制定个性化的干预措施。这种方法可以提高治疗效果并减少不良反应。其次，人工智能在药物研发中通过分析生物数据、预测药物疗效和识别潜在候选药物，加速了新药的发现和开发，从而为患者提供更多针对性的治疗选择。

此外，人工智能在医疗诊断与治疗中起到了重要作用。它能够通过分析医学影像来辅助诊断，预测患者的预后，并协助制定治疗计划。人工智能工具在此过程中提高了准确性、效率和患者护理水平。

还有，人工智能机器人在手术中可以帮助外科医生以更高的精度和控制力执行复杂的手术，提高手术灵活性，减少创伤，并改善患者的治疗效果。

最后，人工智能在医疗保健管理方面的应用，如自动化任务、管理患者记录和优化工作流程，也有助于提高效率、降低成本并增强患者体验。

综上所述，人工智能在个性化医疗方面的贡献包括：提供个性化的治疗干预措施、加速药物研发、提高诊断与治疗的准确性和效率、改善手术结果，以及优化医疗保健管理流程。这些进步共同推动了医疗行业向更加个性化和精准的方向发展。

===== 使用 rewrite 查询运行 RAG =====
从PDF中提取文本...
分割文本...
创建了 13 个文本块
为文本块创建嵌入向量...
向向量存储中添加了 13 个文本块
转换类型: rewrite
原始查询: 人工智能如何为个性化医疗做出贡献？
重写后的查询: 人工智能在实现精准医疗领域中，具体通过哪些技术手段和应用程序促进患者个性化治疗方案的开发和优化？
使用 rewrite 查询的响应:
人工智能为个性化医疗做出了多方面的贡献。首先，人工智能能够通过分析个体患者数据，预测治疗反应，并制定个性化的干预措施，从而实现个性化医疗。这种基于数据的个性化治疗方案可以提高治疗效果，减少不良反应。

其次，人工智能在药物研发方面发挥着重要作用。它通过分析生物数据、预测药物疗效和识别潜在候选药物，加速药物的发现和开发。人工智能系统的应用可以缩短新疗法上市的时间，降低成本，进而促进个性化医疗的